In [3]:
import pandas as pd             # data package
import matplotlib.pyplot as plt # graphics 
import datetime as dt
import numpy as np
from census import Census # This is new...

import requests, io             # internet and input tools  
import zipfile as zf            # zip file tools 
import os  

#import weightedcalcs as wc
#import numpy as np

import pyarrow as pa
import pyarrow.parquet as pq

In [ ]:
date = "2025-03"

my_key = "&key=34e40301bda77077e24c859c6c6c0b721ad73fc7"
# This is my key. I'm nice and I have it posted. If you will be doing more with this
# please get your own key!

In [5]:
end_use = "naics?get=CON_VAL_MO,CTY_CODE,CTY_NAME,SUMMARY_LVL"

url = "https://api.census.gov/data/timeseries/intltrade/imports/" + end_use 
url = url + my_key + "&time==from+2013-01"

r = requests.get(url) 
    
print(r)
    
df = pd.DataFrame(r.json()[1:]) # This then converts it to a dataframe
    # Note that the first entry is the labels

df.columns = r.json()[0]

df["total_imports"] = df["CON_VAL_MO"].astype(float)

df = df[df.SUMMARY_LVL == "DET"]

grp = df.groupby(["CTY_NAME"])

top_products = grp.agg({"total_imports":"sum","CTY_CODE":"first"})

country_list = list(top_products.sort_values(by = "total_imports", ascending = False).CTY_CODE)[0:31]


['TOTAL FOR ALL COUNTRIES','NAFTA','EUROPEAN UNION']

<Response [200]>


['TOTAL FOR ALL COUNTRIES', 'NAFTA', 'EUROPEAN UNION']

In [6]:
country_list[0] = ""

In [7]:
country_list.extend(["0003", "0020"])

In [8]:
country_list

['',
 '5700',
 '2010',
 '1220',
 '5880',
 '4280',
 '5800',
 '5520',
 '5830',
 '5330',
 '4190',
 '4120',
 '4759',
 '4279',
 '4419',
 '5570',
 '5490',
 '3510',
 '5590',
 '4210',
 '5600',
 '5081',
 '5170',
 '4231',
 '4621',
 '4700',
 '3010',
 '4330',
 '6021',
 '4010',
 '5650',
 '0003',
 '0020']

In [9]:
end_use = "hs?get=CTY_NAME,CON_VAL_MO,CAL_DUT_MO,I_COMMODITY,I_COMMODITY_SDESC"

surl = "https://api.census.gov/data/timeseries/intltrade/imports/" + end_use 

surl  = surl + my_key + "&time=" + "from+2013-01" + "&COMM_LVL=HS2" 

for xxx in country_list:
    
    out_file = ".\\data"+ "\\imports\\" + xxx + "data-" + date + ".parquet"
    
    if xxx == "":
        out_file = ".\\data"+ "\\imports\\" + "TOTAL" + "data-" + date + ".parquet"
    
    
    if os.path.exists(out_file):
        
        print("Already have downloaded file")
        
        continue
    
    print(xxx)
    
    url = surl + "&CTY_CODE=" + xxx
    
    if xxx == "":
        url = surl
    
    r = requests.get(url) 
    
    print(r)
    
    foo = pd.DataFrame(r.json()[1:]) # This then converts it to a dataframe
    # Note that the first entry is the labels

    foo.columns = r.json()[0]

    pq.write_table(pa.Table.from_pandas(foo), out_file)


<Response [200]>
5700
<Response [200]>
2010
<Response [200]>
1220
<Response [200]>
5880
<Response [200]>
4280
<Response [200]>
5800
<Response [200]>
5520
<Response [200]>
5830
<Response [200]>
5330
<Response [200]>
4190
<Response [200]>
4120
<Response [200]>
4759
<Response [200]>
4279
<Response [200]>
4419
<Response [200]>
5570
<Response [200]>
5490
<Response [200]>
3510
<Response [200]>
5590
<Response [200]>
4210
<Response [200]>
5600
<Response [200]>
5081
<Response [200]>
5170
<Response [200]>
4231
<Response [200]>
4621
<Response [200]>
4700
<Response [200]>
3010
<Response [200]>
4330
<Response [200]>
6021
<Response [200]>
4010
<Response [200]>
5650
<Response [200]>
0003
<Response [200]>
0020
<Response [200]>


In [10]:
foo

,CTY_NAME,CON_VAL_MO,CAL_DUT_MO,I_COMMODITY,I_COMMODITY_SDESC,time,COMM_LVL,CTY_CODE
0,NAFTA,182150721,14,01,LIVE ANIMALS,2013-01,HS2,0020
1,NAFTA,212532311,156,01,LIVE ANIMALS,2013-02,HS2,0020
2,NAFTA,244602053,57,01,LIVE ANIMALS,2013-03,HS2,0020
3,NAFTA,250009423,199,01,LIVE ANIMALS,2013-04,HS2,0020
4,NAFTA,182211881,50,01,LIVE ANIMALS,2013-05,HS2,0020
...,...,...,...,...,...,...,...,...
14301,NAFTA,213214361,505764,95,"TOYS, GAMES & SPORT EQUIPMENT; PARTS & ACCESSO...",2024-10,HS2,0020
14302,NAFTA,163850473,547369,95,"TOYS, GAMES & SPORT EQUIPMENT; PARTS & ACCESSO...",2024-11,HS2,0020
14303,NAFTA,155446364,840200,95,"TOYS, GAMES & SPORT EQUIPMENT; PARTS & ACCESSO...",2024-12,HS2,0020
14304,NAFTA,213479283,744539,95,"TOYS, GAMES & SPORT EQUIPMENT; PARTS & ACCESSO...",2025-01,HS2,0020


In [11]:
country_list[0] = "TOTAL"

df = pd.DataFrame([])
#this is for all countries...

for xxx in country_list:
    
    file = ".\\data\\imports\\" + xxx + "data-" + date + ".parquet"

    foo = pq.read_table(file).to_pandas()
    
    #recentfile = ".\\data\\imports\\" + xxx + "-" + date + "-" + "data.parquet"
    
    #foo = foo.append(pq.read_table(recentfile).to_pandas())
    
    #pq.write_table(pa.Table.from_pandas(foo), file)
    #this combines the two periods
    
    df = df.append(foo)
            
df.time = pd.to_datetime(df.time, format="%Y-%m")

df["imports"] = df["CON_VAL_MO"].astype(float)

df["duty"] = df["CAL_DUT_MO"].astype(float)

total = df.groupby(["CTY_NAME", "time"]).agg({"imports": "sum", "duty": "sum"})
                   
total.reset_index(inplace = True)
                   
total["I_COMMODITY"] = "00"
total["I_COMMODITY_SDESC"] = "ALL PRODUCTS"
                   
df = df.append(total)

df["itariff"] = 100*df["duty"]/df["imports"]

df.drop(["CON_VAL_MO", "COMM_LVL", "CTY_CODE", "CAL_DUT_MO"], axis = 1, inplace = True)

df["I_COMMODITY_SDESC"] = "HS CODE " + df["I_COMMODITY"] + ", " + df["I_COMMODITY_SDESC"]

df["I_COMMODITY_SDESC"].replace({"HS CODE 00, ALL PRODUCTS": "ALL PRODUCTS"}, inplace = True)

df.set_index(["CTY_NAME","I_COMMODITY_SDESC","time"],inplace = True)

out_file = ".\\data"+ "\\top30-HS2-imports.parquet"

pq.write_table(pa.Table.from_pandas(df), out_file)

In [12]:
df.loc["NAFTA","ALL PRODUCTS"]["2024"]

c:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  """Entry point for launching an IPython kernel.
c:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Indexing a DataFrame with a datetimelike index using a single string to slice the rows, like `frame[string]`, is deprecated and will be removed in a future version. Use `frame.loc[string]` instead.
  """Entry point for launching an IPython kernel.


,I_COMMODITY,imports,duty,itariff
time,,,,
2024-01-01,00,7.123216e+10,138215749.0,0.194036
2024-02-01,00,7.354369e+10,135376122.0,0.184076
2024-03-01,00,7.571552e+10,147118148.0,0.194304
2024-04-01,00,7.773480e+10,144177124.0,0.185473
2024-05-01,00,7.938347e+10,148087188.0,0.186547
2024-06-01,00,7.664725e+10,135528093.0,0.176821
2024-07-01,00,7.733323e+10,144624107.0,0.187014
2024-08-01,00,7.649800e+10,142648253.0,0.186473
2024-09-01,00,7.857459e+10,130604028.0,0.166217


In [13]:
file = ".\\data\\imports\\" + "TOTAL" + "data.parquet"

foo = pq.read_table(file).to_pandas()
    
foo

,CTY_NAME,CON_VAL_MO,CAL_DUT_MO,I_COMMODITY,I_COMMODITY_SDESC,time,COMM_LVL
0,TOTAL FOR ALL COUNTRIES,784536648,4504327,07,EDIBLE VEGETABLES & CERTAIN ROOTS & TUBERS,2013-01,HS2
1,TOTAL FOR ALL COUNTRIES,315892605,2650541,17,SUGARS AND SUGAR CONFECTIONARY,2013-01,HS2
2,TOTAL FOR ALL COUNTRIES,334605750,723046,10,CEREALS,2013-01,HS2
3,TOTAL FOR ALL COUNTRIES,144113171,409478,13,"LAC; GUMS, RESINS & OTHER VEGETABLE SAP & EXTRACT",2013-01,HS2
4,TOTAL FOR ALL COUNTRIES,371169570,2283401,18,COCOA AND COCOA PREPARATIONS,2013-01,HS2
...,...,...,...,...,...,...,...
9991,TOTAL FOR ALL COUNTRIES,794906854,2374814,09,"COFFEE, TEA, MATE & SPICES",2021-06,HS2
9992,TOTAL FOR ALL COUNTRIES,146629956,1172022,10,CEREALS,2021-06,HS2
9993,TOTAL FOR ALL COUNTRIES,186372088,2220530,11,MILLING PRODUCTS; MALT; STARCH; INULIN; WHT GL...,2021-06,HS2
9994,TOTAL FOR ALL COUNTRIES,340184476,3851904,12,"OIL SEEDS ETC.; MISC GRAIN, SEED, FRUIT, PLANT...",2021-06,HS2


In [14]:
foo = pq.read_table(".\\data\\top30-HS2-imports.parquet").to_pandas()

In [15]:
foo.tail()

I_COMMODITY       imports         duty  \
CTY_NAME I_COMMODITY_SDESC time                                                
VIETNAM  ALL PRODUCTS      2024-10-01          00  1.399934e+10  510646925.0   
                           2024-11-01          00  1.277371e+10  416875380.0   
                           2024-12-01          00  1.183467e+10  437928221.0   
                           2025-01-01          00  1.363207e+10  480653058.0   
                           2025-02-01          00  1.273380e+10  405580893.0   

                                        itariff  
CTY_NAME I_COMMODITY_SDESC time                  
VIETNAM  ALL PRODUCTS      2024-10-01  3.647651  
                           2024-11-01  3.263542  
                           2024-12-01  3.700384  
                           2025-01-01  3.525900  
                           2025-02-01  3.185073

In [16]:
foo.loc["TOTAL FOR ALL COUNTRIES", "HS CODE 01, LIVE ANIMALS"]

c:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  """Entry point for launching an IPython kernel.


,I_COMMODITY,imports,duty,itariff
time,,,,
2013-01-01,01,221313094.0,417.0,0.000188
2013-02-01,01,246092109.0,823.0,0.000334
2013-03-01,01,261244392.0,75.0,0.000029
2013-04-01,01,271133926.0,1389.0,0.000512
2013-05-01,01,209785289.0,50.0,0.000024
...,...,...,...,...
2024-10-01,01,536589894.0,88979.0,0.016582
2024-11-01,01,428740780.0,95607.0,0.022299
2024-12-01,01,319316663.0,63516.0,0.019891
